## Photometry on MIRI images

In [ ]:
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS, FITSFixedWarning
from astropy.nddata import Cutout2D
from astropy.table import Table
from regions import EllipsePixelRegion, PixCoord
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import glob
import warnings
import subprocess
import json

warnings.simplefilter("ignore", category=FITSFixedWarning)


Now let's choose a galaxy, read its aperture data and overplot it!

In [ ]:
def load_cutout(file_path, index=1):
    """Loads a FITS cutout file and extracts the data, header, and WCS."""
    try:
        with fits.open(file_path) as hdu:
            data = hdu[index].data
            header = hdu[index].header
            wcs = WCS(header)
        return data, wcs
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return None, None

def adjust_aperture(galaxy_id, filter, survey, obs, output_folder):
    
    # --- Load the FITS table ---
    #table_path =  '/home/bpc/University/master/Red_Cardinal/Flux_Aperture_PSFMatched_AperCorr_old.fits'
    #aperture_table = Table.read(table_path)
    table_path =  '/home/bpc/University/master/Red_Cardinal/aperture_table.csv'
    df = pd.read_csv(table_path)
    
    # --- Select the galaxy by ID ---
    #row = aperture_table[aperture_table['ID'] == galaxy_id][0]
    row = df[df['ID'] == galaxy_id].iloc[0]

    # --- Read in rotation angle of MIRI FITS file ---
    angle_file = '/home/bpc/University/master/Red_Cardinal/rotation_angles.json'
    with open(angle_file, "r") as f:
        angles = json.load(f)
    angle = angles[f"angle_{survey}{obs}"] * u.deg
    
    # --- Read WCS from NIRCam image ---
    nircam_path = f"/home/bpc/University/master/Red_Cardinal/NIRCam/F444W_cutouts/{galaxy_id}_F444W_cutout.fits"
    nircam_data, nircam_wcs = load_cutout(nircam_path)

    # --- Convert NIRCam pixel coordinates to sky ---
    sky_coord = nircam_wcs.pixel_to_world(row['Apr_Xcenter'], row['Apr_Ycenter'])

    # --- Open MIRI cutout image ---
    miri_path = f"/home/bpc/University/master/Red_Cardinal/cutouts/{galaxy_id}_{filter}_cutout_{survey}{obs}.fits"
    miri_data, miri_wcs = load_cutout(miri_path)

    # --- Convert sky coords to MIRI pixel coordinates ---
    miri_x, miri_y = miri_wcs.world_to_pixel(sky_coord)

    # --- Create elliptical region in MIRI pixel space ---
    nircam_scale = 0.031    # arcsec/pixel
    miri_scale = 0.10996    # arcsec/pixel
    scale_factor = nircam_scale / miri_scale
    
    # --- Specify parameters for the ellips ---
    width = row['Apr_A'] * scale_factor
    height = row['Apr_B'] * scale_factor
    theta_rad = row['Apr_Theta'] * u.deg
    theta_rad += angle
    
    # --- Create output file and check if folder exists ---
    os.makedirs(output_folder, exist_ok=True)
    outfile = os.path.join(output_folder, f'{galaxy_id}_{survey}{obs}_aperture_reg')
    # --- Write to DS9-compatible region file ---
    with open(outfile, "w") as fh:
        fh.write("# Region file format: DS9 version 4.1\n")
        fh.write("global color=red dashlist=8 3 width=2 font=\"helvetica 10 normal\" "
                "select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1\n")
        fh.write("image\n")
        fh.write(f"ellipse({miri_x:.2f},{miri_y:.2f},{width:.2f},{height:.2f},{theta_rad:.2f})\n")

    # --- Launch DS9 with the MIRI cutout and the overplotted aperture ---
    #subprocess.run(["ds9", miri_path, "-regions", outfile])

Now let's try and call the function:

In [ ]:
out_dir = "/home/bpc/University/master/Red_Cardinal/photometry/regions/"
adjust_aperture(9871, 'F770W', 'primer', '004', out_dir)

Let's check the table:

In [ ]:
table_path =  '/home/bpc/University/master/Red_Cardinal/Flux_Aperture_PSFMatched_AperCorr_old.fits'

table = Table.read(table_path)
print(table[:5])
table.info()
csv_file = '/home/bpc/University/master/Red_Cardinal/aperture_table.csv'

# Select only the Apr_* columns and ID
selected_cols = table["Apr_A", "Apr_B", "Apr_Xcenter", "Apr_Ycenter", "Apr_Theta", "ID"]

selected_cols.write(csv_file, format="csv", overwrite=True)